In [1]:
import os
import pprint

import pandas as pd
from d2l import mxnet as d2l
from mxnet import np

# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pp = pprint.PrettyPrinter(indent=4)

In [2]:
d2l.DATA_HUB["ml-1M"] = (
    "http://files.grouplens.org/datasets/movielens/ml-1m.zip",
    "c4d9eecfca2ab87c1945afe126590906",
)
data_dir1m = d2l.download_extract("ml-1M")

In [3]:
# @save
def read_data_ml1m():
    iNames = ["user_id", "item_id", "rating", "timestamp"]
    ratings = pd.read_csv(
        os.path.join(data_dir1m, "ratings.dat"), "::", names=iNames, engine="python"
    )
    uNames = ["user_id", "gender", "age", "occupation", "zipcode"]
    users = pd.read_csv(
        os.path.join(data_dir1m, "users.dat"), "::", names=uNames, engine="python"
    )

    # MovieID::Title::Genres
    m_cols = ["item_id", "title", "generes"]
    movies = pd.read_csv(
        os.path.join(data_dir1m, "movies.dat"),
        sep="::",
        names=m_cols,
        usecols=range(3),
        encoding="latin-1",
        engine="python",
    )

    # create one merged DataFrame
    movie_ratings = pd.merge(movies, ratings)
    lens = pd.merge(movie_ratings, users)

    num_users = ratings.user_id.unique().shape[0]
    num_items = ratings.item_id.unique().shape[0]
    return lens, num_users, num_items


rating, num_users, num_items = read_data_ml1m()
rating

,item_id,title,generes,user_id,rating,timestamp,gender,age,occupation,zipcode
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
1000204,3513,Rules of Engagement (2000),Drama|Thriller,5727,4,958489970,M,25,4,92843
1000205,3535,American Psycho (2000),Comedy|Horror|Thriller,5727,2,958489970,M,25,4,92843
1000206,3536,Keeping the Faith (2000),Comedy|Romance,5727,5,958489902,M,25,4,92843
1000207,3555,U-571 (2000),Action|Thriller,5727,3,958490699,M,25,4,92843


In [8]:
rating.head()

young = rating[rating["age"] < 19]
old = rating[rating["age"] > 55]

old.item_id

,item_id,user_id,rating,timestamp,age,occupation
count,38780.000000,38780.000000,38780.000000,3.878000e+04,38780.0,38780.000000
mean,1927.316194,2963.619082,3.766632,9.712856e+08,56.0,9.569856
std,1106.875694,1661.022763,1.062551,9.558104e+06,0.0,6.323344
min,1.000000,2.000000,1.000000,9.567068e+08,56.0,0.000000
25%,1073.000000,1670.000000,3.000000,9.653421e+08,56.0,2.000000
50%,1911.000000,2656.000000,4.000000,9.739963e+08,56.0,13.000000
75%,2919.000000,4193.000000,5.000000,9.748619e+08,56.0,14.000000
max,3952.000000,6038.000000,5.000000,1.046438e+09,56.0,20.000000


In [13]:
e = old.groupby(by='item_id')
def qq(data):
    e = np.array(data, dtype=object)

    f = e[1, 1]
    g = f["P@10"]

    q = []
    for i in e:
        q.append((i[0], np.average(i[1]["P@10"].values)))
    r = pd.DataFrame(q, columns=["rec", "avg"])
    r[["rrec", "config"]] = r.rec.str.split("(", expand=True)
    r.head()
qq(e)

KeyError: <class 'numpy.object_'>

In [4]:
fr = rating[rating["gender"] == "M"]
mr = rating[rating["gender"] == "F"]

In [5]:
rating.describe()
# young = rating[rating['age'] == 'M']

,item_id,user_id,rating,timestamp,age,occupation
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,1.865540e+03,3.024512e+03,3.581564e+00,9.722437e+08,2.973831e+01,8.036138e+00
std,1.096041e+03,1.728413e+03,1.117102e+00,1.215256e+07,1.175198e+01,6.531336e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08,1.000000e+00,0.000000e+00
25%,1.030000e+03,1.506000e+03,3.000000e+00,9.653026e+08,2.500000e+01,2.000000e+00
50%,1.835000e+03,3.070000e+03,4.000000e+00,9.730180e+08,2.500000e+01,7.000000e+00
75%,2.770000e+03,4.476000e+03,4.000000e+00,9.752209e+08,3.500000e+01,1.400000e+01
max,3.952000e+03,6.040000e+03,5.000000e+00,1.046455e+09,5.600000e+01,2.000000e+01


In [4]:
fr.to_csv(r"C:\Projects\RecSys2020\datasets\ml1m\female\ratings.dat", index=False)
mr.to_csv(r"C:\Projects\RecSys2020\datasets\ml1m\male\ratings.dat", index=False)

NameError: name 'fr' is not defined

In [86]:
rating_by_gender = rating.groupby("gender").size().sort_values(ascending=False)[:2]
users_by_gender = rating.groupby("gender").agg({"user_id": pd.Series.nunique})

In [87]:
male_rating = rating[rating["gender"] == "M"]
female_rating = rating[rating["gender"] == "F"]

rating_by_gender

TypeError: _repr_latex_() missing 1 required positional argument: 'self'

In [2]:
d2l.DATA_HUB["ml-100k"] = (
    "http://files.grouplens.org/datasets/movielens/ml-100k.zip",
    "cd4dcac4241c8a4ad7badc7ca635da8a69dddb83",
)


data_dir = d2l.download_extract("ml-100k")
# names = ['user_id', 'item_id', 'rating', 'timestamp']
# data = pd.read_csv(os.path.join(data_dir, 'u.data'), '\t', names=names,
#                   engine='python')

# pass in column names for each CSV
u_cols = ["user_id", "age", "sex", "occupation", "zip_code"]
users = pd.read_csv(
    os.path.join(data_dir, "u.user"), sep="|", names=u_cols, encoding="latin-1"
)

r_cols = ["user_id", "movie_id", "rating", "unix_timestamp"]
ratings = pd.read_csv(
    os.path.join(data_dir, "u.data"), sep="\t", names=r_cols, encoding="latin-1"
)

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ["movie_id", "title", "release_date", "video_release_date", "imdb_url"]
movies = pd.read_csv(
    os.path.join(data_dir, "u.item"),
    sep="|",
    names=m_cols,
    usecols=range(5),
    encoding="latin-1",
)

# create one merged DataFrame
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)

In [7]:
lens.describe()

lens['user_id'].unique().size

943

In [8]:
young = lens[lens["age"] < 19]
old = lens[lens["age"] > 55]

In [12]:
young['user_id'].unique().size

54

In [ ]:
bins = [0, 18, 25, 35, 45, 50, 56, 200]
labels = ["Under 18", "18-24", "25-34", "35-44", "45-49", "50-55", "56+"]
lens["AgeGroup"] = pd.cut(lens["age"], bins=bins, labels=labels, right=False)


lens.head(150)
# np.array(e[1].keys())

In [13]:
def groupby2(by):
    r = lens.groupby(by).size()
    u = lens.groupby(by).agg({"user_id": pd.Series.nunique})
    ppp = pd.DataFrame(
        data=[r.tolist(), np.array(u)],
        index=["ratings", "users"],
        columns=lens.groupby(by).size().keys().tolist(),
    )
    return ppp


e = []
for i in ["sex", "AgeGroup", "occupation"]:
    e.append(groupby2(i))

lens.head()

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,unix_timestamp,age,sex,occupation,zip_code,AgeGroup
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,60,M,retired,95076,56+
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,887737890,60,M,retired,95076,56+
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),308,4,887739608,60,M,retired,95076,56+
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,308,4,887738847,60,M,retired,95076,56+
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),308,5,887736696,60,M,retired,95076,56+


In [14]:
e[1].T

,ratings,users
Under 18,2491,[[[[36.]]]]
18-24,24060,[[[[198.]]]]
25-34,35444,[[[[310.]]]]
35-44,19591,[[[[194.]]]]
45-49,6890,[[[[80.]]]]
50-55,6534,[[[[73.]]]]
56+,4990,[[[[52.]]]]


,movie_id,video_release_date,user_id,rating,unix_timestamp,age
count,4710.000000,0.0,4710.000000,4710.000000,4.710000e+03,4710.000000
mean,461.844161,NaN,569.222930,3.552229,8.837090e+08,16.469427
std,351.680234,NaN,230.513726,1.213348,4.854273e+06,2.057524
min,1.000000,NaN,30.000000,1.000000,8.747282e+08,7.000000
25%,186.250000,NaN,434.000000,3.000000,8.801678e+08,15.000000
50%,339.000000,NaN,592.000000,4.000000,8.829610e+08,17.000000
75%,721.000000,NaN,674.000000,5.000000,8.881800e+08,18.000000
max,1680.000000,NaN,925.000000,5.000000,8.930908e+08,18.000000


In [41]:
def save_csv(data):
    values = data["lens"].sort_values(by="user_id")

    try:
        os.mkdir(data["path"])
    except OSError:
        print("Creation of the directory %s failed" % data["path"])
    else:
        print("Successfully created the directory %s " % data["path"])

    csv_path = os.path.join(data["path"], "data.txt")
    values.to_csv(
        csv_path,
        index=False,
        header=False,
        columns=["user_id", "movie_id", "rating", "unix_timestamp"],
        sep="\t",
    )

young = lens[lens["age"] < 19]
old = lens[lens["age"] > 55]

path = os.path.join(r"C:\Projects\RecSys2020\datasets\ml100k", "young")
save_csv({"lens": young, "path": path})

path = os.path.join(r"C:\Projects\RecSys2020\datasets\ml100k", "old")
save_csv({"lens": old, "path": path})

print(young['movie_id'].unique().size, old['movie_id'].unique().size)

Creation of the directory C:\Projects\RecSys2020\datasets\ml100k\young failed
Creation of the directory C:\Projects\RecSys2020\datasets\ml100k\old failed
1012 1042


In [40]:
young['movie_id'].unique().size


1012

In [ ]:
data_dirM = r"C:\Projects\RecSys2020\datasets\ml100k\male\\"
data_dirF = r"C:\Projects\RecSys2020\datasets\ml100k\female\\"

fr = lens[lens["sex"] == "F"].sort_values(by="user_id")
mr = lens[lens["sex"] == "M"].sort_values(by="user_id")

rating_columns = ["user_id", "movie_id", "rating", "unix_timestamp"]
fname = "data.txt"
params = dict(index=False, header=False, columns=rating_columns, sep="\t")
fr.to_csv(os.path.join(data_dirF, fname), **params)
mr.to_csv(os.path.join(data_dirM, fname), **params)

In [5]:
most_rated = lens.groupby("title").size().sort_values(ascending=False)[:5]
most_rated

title
Star Wars (1977)             583
Contact (1997)               509
Fargo (1996)                 508
Return of the Jedi (1983)    507
Liar Liar (1997)             485
dtype: int64

In [6]:
lens.title.value_counts()[:5]
# SELECT title, count(1)
# FROM lens
# GROUP BY title
# ORDER BY 2 DESC
# LIMIT 25;

Star Wars (1977)             583
Contact (1997)               509
Fargo (1996)                 508
Return of the Jedi (1983)    507
Liar Liar (1997)             485
Name: title, dtype: int64

In [8]:
print(lens.user_id.unique().shape[0])
print(lens.movie_id.unique().shape[0])
print(lens.sex.unique().shape[0])
print(lens.occupation.unique().shape[0])
print(lens.zip_code.unique().shape[0])

##http://www.gregreda.com/2013/10/26/using-pandas-on-the-movielens-dataset/

943
1682
2
21
795


rating_by_gender = lens.groupby("sex").size().sort_values(ascending=False)[:2]
users_by_gender = lens.groupby("sex").agg({"user_id": pd.Series.nunique})
# .size().sort_values(ascending=False)[:2]

In [55]:
print(
    "users_by_gender",
    users_by_gender["user_id"]["M"] / users_by_gender.sum()[0],
    users_by_gender["user_id"]["F"] / users_by_gender.sum()[0],
)

print(
    "rating_by_gender",
    rating_by_gender["M"] / rating_by_gender.sum(),
    rating_by_gender["F"] / rating_by_gender.sum(),
)

users_by_gender 0.7104984093319194 0.28950159066808057
rating_by_gender 0.7426 0.2574


In [50]:
users_by_gender["user_id"].sum()

100000

In [52]:
users_by_gender.sum()[0]

943

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch
from sklearn.model_selection import (
    GroupKFold,
    ShuffleSplit,
    StratifiedKFold,
    StratifiedShuffleSplit,
    TimeSeriesSplit,
)

np.random.seed(1338)
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm
n_splits = 4

In [ ]:
y = np.array(lens["sex"])
groups = ["red", "blue"]


def visualize_groups(classes, groups, name):
    # Visualize dataset groups
    fig, ax = plt.subplots()
    ax.scatter(
        range(len(groups)),
        [0.5] * len(groups),
        c=groups,
        marker="_",
        lw=50,
        cmap=cmap_data,
    )
    ax.scatter(
        range(len(groups)),
        [3.5] * len(groups),
        c=classes,
        marker="_",
        lw=50,
        cmap=cmap_data,
    )
    ax.set(
        ylim=[-1, 5],
        yticks=[0.5, 3.5],
        yticklabels=["Data\ngroup", "Data\nclass"],
        xlabel="Sample index",
    )


visualize_groups(y, groups, "no groups")
plt.show()

In [6]:
plt.show()

NameError: name 'plt' is not defined

In [7]:
lens["sex"].unique()
%pylab inline

NameError: name 'lens' is not defined

In [238]:
def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2,
        )

    # Plot the data classes and groups at the end
    ax.scatter(
        range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data
    )

    ax.scatter(
        range(len(X)), [ii + 2.5] * len(X), c=group, marker="_", lw=lw, cmap=cmap_data
    )

    # Formatting
    yticklabels = list(range(n_splits)) + ["class", "group"]
    ax.set(
        yticks=np.arange(n_splits + 2) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
        ylim=[n_splits + 2.2, -0.2],
        xlim=[0, 100],
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)
    return ax

In [239]:
fig, ax = plt.subplots()
cv = StratifiedKFold(n_splits)
lens["sexcode"] = pd.factorize(lens["sex"])[0] + 1
groups = np.array(lens["sexcode"])
plot_cv_indices(cv, lens, lens["sexcode"], groups, ax, n_splits)

ax.legend(
    [Patch(color=cmap_cv(0.8)), Patch(color=cmap_cv(0.02))],
    ["Testing set", "Training set"],
    loc=(1.02, 0.8),
)
plt.tight_layout()
fig.subplots_adjust(right=0.7)
plt.show()

In [124]:
pd.factorize(lens["sex"])[0] + 1

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [177]:
x = lens["sexcode"].where(lens["sexcode"] == 2)
print(lens["sexcode"].values.count("M"))

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [90]:
groups = np.hstack([[ii] * 10 for ii in range(10)])
print(groups)
print(len(groups))

[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9]
100


In [182]:
lens[lens["sex"] == "F"].movie_id

524         1
525         2
526         3
527         4
528         5
         ... 
99974     682
99975     873
99976     877
99977     886
99978    1527
Name: movie_id, Length: 25740, dtype: int64

In [166]:
lens["sex"].values
unique, counts = np.unique(lens["sex"].values, return_counts=True)

print(np.asarray((unique, counts)))

[['F' 'M']
 [25740 74260]]


In [172]:
j = 0
y = 0
for i in pd.factorize(lens["sex"])[0] + 1:
    if i == 2:
        j += 1
    else:
        y += 1
print(j)
print(y)

print(j + y)

25740
74260
100000


In [185]:
import numpy as np
from sklearn.model_selection import KFold

X = ["a", "b", "c", "d"]
kf = KFold(n_splits=2)
for train, test in kf.split(lens):
    print("%s %s" % (len(train), len(test)))

50000 50000
50000 50000


In [226]:
# Generate the class/group data
n_points = 100000
X = np.array(lens["rating"])  # np.random.randn(100, 10)
y = np.array(lens["sexcode"]).T
groups = pd.factorize(lens["age"])[0] + 1
ageGroups = pd.factorize(lens["age"])[0] + 1
occupationGroups = pd.factorize(lens["occupation"])[0] + 1

In [240]:
def visualize_groups(classes, groups, name):
    # Visualize dataset groups
    fig, ax = plt.subplots()
    ax.scatter(
        range(len(groups)),
        [0.5] * len(groups),
        c=groups,
        marker="_",
        lw=50,
        cmap=cmap_data,
    )
    ax.scatter(
        range(len(groups)),
        [3.5] * len(groups),
        c=classes,
        marker="_",
        lw=50,
        cmap=cmap_data,
    )
    ax.set(
        ylim=[-1, 5],
        yticks=[0.5, 3.5],
        yticklabels=["Data\ngroup", "Data\nclass"],
        xlabel="Sample index",
    )


visualize_groups(y, groups, "no groups")
ax.legend(
    [Patch(color=cmap_cv(0.8)), Patch(color=cmap_cv(0.02))],
    ["Testing set", "Training set"],
    loc=(1.02, 0.8),
)
plt.tight_layout()
fig.subplots_adjust(right=0.7)

plt.show()

ValueError: 'c' argument must be a color, a sequence of colors, or a sequence of numbers, not ['M' 'M' 'M' ... 'M' 'M' 'M']

In [196]:
np.hstack([[ii] * 10 for ii in range(10)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

In [233]:
def plot_cv_indices(cv, X, y, group, group2, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2,
        )

    # Plot the data classes and groups at the end
    ax.scatter(
        range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data
    )

    ax.scatter(
        range(len(X)), [ii + 2.5] * len(X), c=group, marker="_", lw=lw, cmap=cmap_data
    )

    ax.scatter(
        range(len(X)), [ii + 3.5] * len(X), c=group2, marker="_", lw=lw, cmap=cmap_data
    )

    # Formatting
    yticklabels = list(range(n_splits)) + ["sex", "age", "occupation"]
    ax.set(
        yticks=np.arange(n_splits + 3) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
        ylim=[n_splits + 3.2, -0.2],
        xlim=[0, 100000],
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)
    return ax


cvs = [
    KFold,
    GroupKFold,
    ShuffleSplit,
    StratifiedKFold,
    # GroupShuffleSplit,
    StratifiedShuffleSplit,
    TimeSeriesSplit,
]

for cv in cvs:
    this_cv = cv(n_splits=n_splits)
    fig, ax = plt.subplots(figsize=(6, 3))
    plot_cv_indices(this_cv, X, y, ageGroups, occupationGroups, ax, n_splits)

    ax.legend(
        [Patch(color=cmap_cv(0.8)), Patch(color=cmap_cv(0.02))],
        ["Testing set", "Training set"],
        loc=(1.02, 0.8),
    )
    # Make the legend fit
    plt.tight_layout()
    fig.subplots_adjust(right=0.7)
plt.show()

In [221]:
len(X)

100000

In [223]:
def plot_cv_indices(cv, X, y, group, group2, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2,
        )

    # Plot the data classes and groups at the end
    ax.scatter(
        range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data
    )

    ax.scatter(
        range(len(X)), [ii + 2.5] * len(X), c=group, marker="_", lw=lw, cmap=cmap_data
    )

    ax.scatter(
        range(len(X)), [ii + 3.5] * len(X), c=group2, marker="_", lw=lw, cmap=cmap_data
    )

    # Formatting
    yticklabels = list(range(n_splits)) + ["sex", "age", "occupation"]
    ax.set(
        yticks=np.arange(n_splits + 3) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
        ylim=[n_splits + 3.2, -0.2],
        xlim=[0, 100000],
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)
    return ax


cvs1 = [KFold, GroupKFold, ShuffleSplit]
cvs2 = [
    StratifiedKFold,
    # GroupShuffleSplit,
    StratifiedShuffleSplit,
    TimeSeriesSplit,
]

fig, axs = plt.subplots(2, 3)

for i, cv in enumerate(cvs1):
    this_cv = cv(n_splits=n_splits)
    ax = axs[0, i]
    plot_cv_indices(this_cv, X, y, ageGroups, occupationGroups, ax, n_splits)


for i, cv in enumerate(cvs2):
    this_cv = cv(n_splits=n_splits)
    ax = axs[1, i]
    plot_cv_indices(this_cv, X, y, ageGroups, occupationGroups, ax, n_splits)

ax.legend(
    [Patch(color=cmap_cv(0.8)), Patch(color=cmap_cv(0.02))],
    ["Testing set", "Training set"],
    loc=(1.02, 0.8),
)
# Make the legend fit
fig.set_size_inches(6, 12)

plt.tight_layout()
fig.subplots_adjust(right=0.7)
plt.show()